## Load the data set

In [54]:
/read the csv

hfi : ("*FFFFFFFFFFFFF";enlist csv)0: `:./data/edhec_hedgefundindices.csv

/ Devide by 100

hfi : (select date from hfi)^(delete date from hfi) % 100

5#hfi

date         Convertible Arbitrage CTA Global Distressed Securities Emerging ..
-----------------------------------------------------------------------------..
"31/01/1997" 0.0119                0.0393     0.0178                0.0791   ..
"28/02/1997" 0.0123                0.0298     0.0122                0.0525   ..
"31/03/1997" 0.0078                -0.0021    -0.0012               -0.012   ..
"30/04/1997" 0.0086                -0.017     0.003                 0.0119   ..
"31/05/1997" 0.0156                -0.0015    0.0233                0.0315   ..


In [55]:
/ Convert the date type to month

/it can handle string and list type

str2month:{if[10h=type x;x:enlist x]; `month$"D"$"."sv'reverse@/:"/"vs/:x}

In [56]:
str2month hfi.date

1997.01 1997.02 1997.03 1997.04 1997.05 1997.06 1997.07 1997.08 1997.09 1997...


In [57]:
/update the date column as month type

update date: str2month date from `hfi

`hfi


In [58]:
meta hfi

c                     | t f a
----------------------| -----
date                  | m    
Convertible Arbitrage | f    
CTA Global            | f    
Distressed Securities | f    
Emerging Markets      | f    
Equity Market Neutral | f    
Event Driven          | f    
Fixed Income Arbitrage| f    
Global Macro          | f    
Long/Short Equity     | f    
Merger Arbitrage      | f    
Relative Value        | f    
Short Selling         | f    
Funds Of Funds        | f    


In [59]:
/ Remove the special char and space and replace with underscore

hfi : (`$(ssr/[;(" ";"/");"_"] each string cols hfi)) xcol hfi;

meta hfi

c                     | t f a
----------------------| -----
date                  | m    
Convertible_Arbitrage | f    
CTA_Global            | f    
Distressed_Securities | f    
Emerging_Markets      | f    
Equity_Market_Neutral | f    
Event_Driven          | f    
Fixed_Income_Arbitrage| f    
Global_Macro          | f    
Long_Short_Equity     | f    
Merger_Arbitrage      | f    
Relative_Value        | f    
Short_Selling         | f    
Funds_Of_Funds        | f    


## Skewness

Intuitively, a negative skew means that you get more negative returns than you would have expected if the returns were distributed like the normal distribution.

Another way of thinking about it is if that returns are normally distributed, the mean and the median would be very close.

However, if they are negatively skewed, the expected value i.e. the mean is less than the median. If they are positively skewed, the expected value (again, the mean) is greater than the median.

In [73]:
hfi_skw:([] hfi: key mn:avg (delete date from hfi); mean: value mn; median: value med (delete date from hfi))

hfi_skw

hfi                    mean        median 
------------------------------------------
Convertible_Arbitrage  0.005508365 0.0065 
CTA_Global             0.004073764 -0.0204
Distressed_Securities  0.006946008 0.0091 
Emerging_Markets       0.006253232 -0.0133
Equity_Market_Neutral  0.004498099 0.0023 
Event_Driven           0.006344487 0.0058 
Fixed_Income_Arbitrage 0.004365019 0.0061 
Global_Macro           0.005403042 -0.0061
Long_Short_Equity      0.006330798 0.0033 
Merger_Arbitrage       0.005355894 0.0024 
Relative_Value         0.005792015 0.0055 
Short_Selling          -0.00170076 -0.0123
Funds_Of_Funds         0.004262357 0.0001 


In [74]:
update skewness_type:(`positive`negative) mean < median from `hfi_skw;
hfi_skw

hfi                    mean        median  skewness_type
--------------------------------------------------------
Convertible_Arbitrage  0.005508365 0.0065  negative     
CTA_Global             0.004073764 -0.0204 positive     
Distressed_Securities  0.006946008 0.0091  negative     
Emerging_Markets       0.006253232 -0.0133 positive     
Equity_Market_Neutral  0.004498099 0.0023  positive     
Event_Driven           0.006344487 0.0058  positive     
Fixed_Income_Arbitrage 0.004365019 0.0061  negative     
Global_Macro           0.005403042 -0.0061 positive     
Long_Short_Equity      0.006330798 0.0033  positive     
Merger_Arbitrage       0.005355894 0.0024  positive     
Relative_Value         0.005792015 0.0055  positive     
Short_Selling          -0.00170076 -0.0123 positive     
Funds_Of_Funds         0.004262357 0.0001  positive     


Now, let's develop the code to compute the skewness of a series of numbers.


$$ S(R) = \frac{E[ (R-E(R))^3 ]}{\sigma_R^3} $$


In [80]:
skewness: {[r] (avg((r-avg r) xexp 3)) % (dev r) xexp 3}

In [84]:
skewness_value: value (skewness each flip (delete date from hfi))

update skewness: skewness_value from `hfi_skw;

hfi_skw

hfi                    mean        median  skewness_type skewness  
-------------------------------------------------------------------
Convertible_Arbitrage  0.005508365 0.0065  negative      -2.639592 
CTA_Global             0.004073764 -0.0204 positive      0.1736986 
Distressed_Securities  0.006946008 0.0091  negative      -1.300842 
Emerging_Markets       0.006253232 -0.0133 positive      -1.167067 
Equity_Market_Neutral  0.004498099 0.0023  positive      -2.124435 
Event_Driven           0.006344487 0.0058  positive      -1.409154 
Fixed_Income_Arbitrage 0.004365019 0.0061  negative      -3.94032  
Global_Macro           0.005403042 -0.0061 positive      0.9829219 
Long_Short_Equity      0.006330798 0.0033  positive      -0.3902268
Merger_Arbitrage       0.005355894 0.0024  positive      -1.320083 
Relative_Value         0.005792015 0.0055  positive      -1.81547  
Short_Selling          -0.00170076 -0.0123 positive      0.7679748 
Funds_Of_Funds         0.004262357 0.0001  posit

# Kurtosis

The kurtosis simply means the fatness of the distribution.

value < 3 --> Thinner tails

value > 3 --> Fatter tails

$$ K(R) = \frac{E[ (R-E(R))^4 ]}{\sigma_R^4} $$

This is almost similar to skewness. So edit the power of 4 part.


In [85]:
/calculate the kurtosis

kurtosis: {[r] (avg ((r-avg r) xexp 4))%(dev r) xexp 4}

In [88]:
kurtosis_value: kurtosis each flip delete date from hfi
kurtosis_value

Convertible_Arbitrage | 23.28083
CTA_Global            | 2.95296
Distressed_Securities | 7.889983
Emerging_Markets      | 9.250788
Equity_Market_Neutral | 17.21856
Event_Driven          | 8.035828
Fixed_Income_Arbitrage| 29.8422
Global_Macro          | 5.741679
Long_Short_Equity     | 4.523893
Merger_Arbitrage      | 8.73895
Relative_Value        | 12.12121
Short_Selling         | 6.117772
Funds_Of_Funds        | 7.070153


In [89]:
update kurtosis: value kurtosis_value from `hfi_skw;

hfi_skw

hfi                    mean        median  skewness_type skewness   kurtosis
----------------------------------------------------------------------------
Convertible_Arbitrage  0.005508365 0.0065  negative      -2.639592  23.28083
CTA_Global             0.004073764 -0.0204 positive      0.1736986  2.95296 
Distressed_Securities  0.006946008 0.0091  negative      -1.300842  7.889983
Emerging_Markets       0.006253232 -0.0133 positive      -1.167067  9.250788
Equity_Market_Neutral  0.004498099 0.0023  positive      -2.124435  17.21856
Event_Driven           0.006344487 0.0058  positive      -1.409154  8.035828
Fixed_Income_Arbitrage 0.004365019 0.0061  negative      -3.94032   29.8422 
Global_Macro           0.005403042 -0.0061 positive      0.9829219  5.741679
Long_Short_Equity      0.006330798 0.0033  positive      -0.3902268 4.523893
Merger_Arbitrage       0.005355894 0.0024  positive      -1.320083  8.73895 
Relative_Value         0.005792015 0.0055  positive      -1.81547   12.12121